Find the median follower count of users that have joined between 2015 and 2020.

In [0]:
%run "/Users/sarfaraj_ahmed@outlook.com/Clean"

In [0]:
df_user.createOrReplaceTempView("user")
df_pin.createOrReplaceTempView("pin")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW users_joined AS
SELECT
  EXTRACT(YEAR FROM u.date_joined) AS post_year,
  COUNT(*) AS number_users_joined,
  p.follower_count
FROM
  user u
INNER JOIN 
  pin p
ON
  u.ind = p.ind
WHERE
  EXTRACT(YEAR FROM u.date_joined) BETWEEN 2015 AND 2020
  AND
  p.follower_count IS NOT NULL 
  AND 
  p.follower_count != 0
GROUP BY
  EXTRACT(YEAR FROM u.date_joined), p.follower_count;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW users_and_followers AS
SELECT
  post_year,
  follower_count,
  COUNT(*) OVER (PARTITION BY post_year) AS total_count,
  ROW_NUMBER() OVER (PARTITION BY post_year ORDER BY follower_count) AS row_num
FROM
  users_joined
GROUP BY
  post_year, follower_count;

In [0]:
%sql
SELECT
  post_year,
  CASE
        WHEN total_count % 2 != 0 THEN MAX(follower_count)
        ELSE AVG(follower_count)
    END AS median_follower_count
FROM
  users_and_followers
WHERE 
    row_num IN ((total_count + 1) / 2, total_count / 2 + 1)
GROUP BY
  post_year, total_count
ORDER BY
  post_year ASC;